# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [5]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [6]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [8]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].rename(columns={'sales_ymd': 'sales_date'}).head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [18]:
df_receipt[df_receipt['customer_id']=='CS018205000001']

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
36,20180911,1536624000,S13018,1122,2,CS018205000001,P071401012,1,2200
9843,20180414,1523664000,S13018,1142,2,CS018205000001,P060104007,6,600
21110,20170614,1497398400,S13018,1112,2,CS018205000001,P050206001,5,990
27673,20170614,1497398400,S13018,1112,1,CS018205000001,P060702015,1,108
27840,20190216,1550275200,S13018,1152,2,CS018205000001,P071005024,1,102
28757,20180414,1523664000,S13018,1142,1,CS018205000001,P071101002,1,278
39256,20190226,1551139200,S13018,1132,2,CS018205000001,P070902035,1,168
58121,20190924,1569283200,S13018,1102,1,CS018205000001,P060805001,1,495
68117,20190226,1551139200,S13018,1132,1,CS018205000001,P071401020,1,2200
72254,20180911,1536624000,S13018,1122,1,CS018205000001,P071401005,1,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [7]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id=="CS018205000001" and amount >= 1000')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [10]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']].query('customer_id == "CS018205000001" and (amount >= 1000 or quantity >=5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [12]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001" and amount >= 1000 and amount <= 2000')

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [13]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id=="CS018205000001" & product_cd != "P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [3]:
df_store.query('prefecture_cd !="13" & floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [13]:
df_store.query("store_cd.str.startswith('S14')",engine="python")

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [28]:
df_customer.query("customer_id.str.match('.+1$')",engine="python").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [29]:
df_store.query("address.str.contains('横浜市')",engine="python")

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [37]:
df_customer.query("status_cd.str.match('^[A-F]')",engine="python").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [39]:
df_customer.query("status_cd.str.contains('[1-9]$')",engine="python").head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [41]:
df_customer.query("status_cd.str.match('^[A-Z].*[1-9]$')",engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [43]:
df_store.query("tel_no.str.match('\d{3}-\d{3}-\d{4}')",engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [44]:
df_customer.sort_values('birth_day',ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [46]:
df_customer.sort_values('birth_day',ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [9]:
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']] 
                     ,df_receipt['amount'].rank(method='min', 
                                                ascending=False)], axis=1)
df_tmp.columns=['customer_id','amount','ranking']
df_tmp.sort_values("ranking",ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [25]:
receipt_rank = pd.concat([df_receipt[['customer_id','amount']],
          df_receipt['amount'].rank(method="first",ascending=False)],axis=1)
receipt_rank.columns = ["customer_id","amount","ranking"]
receipt_rank.sort_values('ranking',ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [27]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [31]:
len(df_receipt['customer_id'].unique())

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [39]:
df_receipt.groupby('store_cd').agg({'amount':'sum','quantity':'sum'}).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [41]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'max'}).reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [42]:
df_receipt.groupby('customer_id').sales_ymd.min().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [51]:
sales_group_receipt = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
sales_group_receipt.columns = ['customer_id','sales_ymd_max','sales_ymd_min']
sales_group_receipt.query('sales_ymd_max != sales_ymd_min').head(10)

,customer_id,sales_ymd_max,sales_ymd_min
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [59]:
df_receipt.groupby('store_cd').amount.mean().reset_index().sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [60]:
df_receipt.groupby('store_cd').amount.median().reset_index()\
                                .sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [66]:
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index()

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [67]:
df_receipt.groupby('store_cd').amount.var().reset_index()\
                                .sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,441863.252526
31,S14011,306442.242432
42,S14034,297068.392740
5,S13001,295558.842618
12,S13015,295427.197086


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [68]:
df_receipt.groupby('store_cd').amount.std().reset_index().sort_values('amount',ascending=False).head(5)

,store_cd,amount
28,S13052,664.727954
31,S14011,553.572256
42,S14034,545.039808
5,S13001,543.653237
12,S13015,543.532149


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [70]:
df_receipt.amount.quantile([0,0.25,0.5,0.75,1])

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [71]:
df_receipt.groupby('store_cd').amount.mean().reset_index().query('amount >=330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [81]:
df_receipt.query("customer_id.str.match('^(?!Z).*') ",engine='python').groupby('customer_id')\
.amount.sum().mean()

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [5]:
entire_mean_value = df_receipt.query("not customer_id.str.startswith('Z')",engine='python').\
                groupby('customer_id').amount.sum().mean()
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_amount_sum[df_amount_sum['amount'] >= entire_mean_value].head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [3]:
mean = df_receipt.query("not customer_id.str.startswith('Z')",engine='python')\
        .groupby('customer_id').amount.sum().mean()
df_receipt.query('customer_id >= mean')

2547.742234529256

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [5]:
pd.merge(df_product
         , df_category[['category_small_cd','category_small_name']]
         , how='inner', on='category_small_cd').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [18]:
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_customer_sum = df_customer.query("not customer_id.str.startswith('Z')",engine='python')[df_customer['gender'] != '女性']
pd.merge(df_amount_sum,df_customer_sum['customer_id'],how='right',on='customer_id').fillna(0).head(10)

,customer_id,amount
0,CS037613000071,0.0
1,CS020401000016,0.0
2,CS029403000008,0.0
3,CS015804000004,0.0
4,CS007403000016,0.0
5,CS040702000012,0.0
6,CS035415000029,7504.0
7,CS040314000027,348.0
8,CS024303000010,0.0
9,CS003802000009,0.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [38]:
#df_most_saled = df_receipt.sort_values('amount',ascending=False).head(20)
df_ignore_z_customer = df_receipt.query("not customer_id.str.startswith('Z')",engine='python')

df_most_saled_amount = df_ignore_z_customer.groupby('customer_id').amount.sum().reset_index().sort_values('amount',ascending=False).head(20)

df_date_saled_count = df_receipt[~df_receipt.duplicated(subset=['customer_id', 'sales_ymd'])].groupby('customer_id').sales_ymd.count().reset_index().\
sort_values('sales_ymd',ascending=False).head(20)

pd.merge(df_most_saled_amount,df_date_saled_count,how='outer',on='customer_id')

,customer_id,amount,sales_ymd
0,CS017415000097,23086.0,20.0
1,CS015415000185,20153.0,22.0
2,CS031414000051,19202.0,19.0
3,CS028415000007,19127.0,21.0
4,CS001605000009,18925.0,NaN
5,CS010214000010,18585.0,22.0
6,CS016415000141,18372.0,20.0
7,CS006515000023,18372.0,NaN
8,CS011414000106,18338.0,NaN
9,CS038415000104,17847.0,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [48]:
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()

df_store_tmp['key']= 0
df_product_tmp['key']= 0

len(pd.merge(df_store_tmp,df_product_tmp,how='outer',on='key'))

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [53]:
df_amount_day_by_day = df_receipt.groupby('sales_ymd').sum().reset_index()
df_amount_day_by_day['diff_amount'] = df_amount_day_by_day['amount'].diff()
df_amount_day_by_day.head(10)

,sales_ymd,sales_epoch,receipt_no,receipt_sub_no,quantity,amount,diff_amount
0,20170101,132007363200,90398,133,90,33723,NaN
1,20170102,127565107200,93222,129,87,24165,-9558.0
2,20170103,133506144000,95910,133,90,27503,3338.0
3,20170104,151315776000,111174,155,104,36165,8662.0
4,20170105,124620249600,88858,127,86,37830,1665.0
5,20170106,117209203200,87378,119,80,32387,-5443.0
6,20170107,127602259200,92472,132,87,23415,-8972.0
7,20170108,140964192000,96460,144,95,24737,1322.0
8,20170109,143940240000,96744,147,98,26718,1981.0
9,20170110,105364454400,72272,106,71,20143,-6575.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [59]:
df_sales_amount_by_date = df_receipt.groupby('sales_ymd').amount.sum().reset_index()

for i in range(1,4):
    if i == 1:
        df_lag = pd.concat([df_sales_amount_by_date, 
                            df_sales_amount_by_date.shift(i)],axis=1)
    else:
        df_lag = df_lag.append(pd.concat([df_sales_amount_by_date, 
                                          df_sales_amount_by_date.shift(i)],
                                          axis=1))
df_lag.columns = ['sales_ymd', 'amount', 'lag_ymd', 'lag_amount']
df_lag.dropna().sort_values(['sales_ymd','lag_ymd']).head(10)

,sales_ymd,amount,lag_ymd,lag_amount
1,20170102,24165,20170101.0,33723.0
2,20170103,27503,20170101.0,33723.0
2,20170103,27503,20170102.0,24165.0
3,20170104,36165,20170101.0,33723.0
3,20170104,36165,20170102.0,24165.0
3,20170104,36165,20170103.0,27503.0
4,20170105,37830,20170102.0,24165.0
4,20170105,37830,20170103.0,27503.0
4,20170105,37830,20170104.0,36165.0
5,20170106,32387,20170103.0,27503.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [23]:
df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
df_sales_summary = pd.pivot_table(df_tmp, index='era', columns='gender_cd', 
                                  values='amount', aggfunc='sum').reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [24]:
df_sales_summary = df_sales_summary.set_index('era').stack().reset_index().replace({'male':'00','female':'01','unknown':'99'})\
.rename(columns={'level_1':'gender_cd', 0: 'amount'})
df_sales_summary

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [32]:
df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')
df_customer[['customer_id','birth_day']]

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [3]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['application_date'])],axis=1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [2]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_ymd'])],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,1970-01-01 00:00:00.020181103
1,1132,2,1970-01-01 00:00:00.020181118
2,1102,1,1970-01-01 00:00:00.020170712
3,1132,1,1970-01-01 00:00:00.020190205
4,1102,2,1970-01-01 00:00:00.020180821
5,1112,1,1970-01-01 00:00:00.020190605
6,1102,2,1970-01-01 00:00:00.020181205
7,1102,1,1970-01-01 00:00:00.020190922
8,1112,2,1970-01-01 00:00:00.020170504
9,1102,1,1970-01-01 00:00:00.020191010


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [7]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [12]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch']).dt.year],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,1970
1,1132,2,1970
2,1102,1,1970
3,1132,1,1970
4,1102,2,1970
5,1112,1,1970
6,1102,2,1970
7,1102,1,1970
8,1112,2,1970
9,1102,1,1970


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、「月は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [16]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],\
           pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.month],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,7
3,1132,1,2
4,1102,2,8
5,1112,1,6
6,1102,2,12
7,1102,1,9
8,1112,2,5
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、「日」は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [9]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
                      pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%d')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [22]:
df_sales_amount = df_receipt.query("not customer_id.str.startswith('Z')",engine='python')\
                    .groupby('customer_id').amount.sum().reset_index()
df_sales_amount['sales_flag'] = df_sales_amount['amount'].apply(lambda x: 0 if x <= 2000 else 1)
df_sales_amount[['customer_id','amount','sales_flag']].head(10)

,customer_id,amount,sales_flag
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [34]:
df_tokyo_customer = df_customer[['customer_id','postal_cd']].copy()
df_tokyo_customer['postal_flag'] = df_customer['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)
pd.merge(df_tokyo_customer,df_receipt,how='inner',on="customer_id").groupby('postal_cd').agg({'customer_id':'nunique'}).reset_index()

,postal_cd,customer_id
0,101-0038,1
1,101-0041,1
2,104-0053,2
3,106-0031,1
4,107-0062,1
...,...,...
918,332-0015,5
919,332-0023,1
920,352-0032,16
921,359-0027,1


---
> P-054: 顧客データフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [40]:
import re

address_code = {'埼玉県':11,'千葉県':12,'東京都':13,'神奈川県':14}
df_tmp = df_customer[['customer_id','address']].copy()
df_tmp['address_flag'] = df_tmp['address'].apply(lambda x: address_code[re.match('.+?[県都]',x).group()])
df_tmp.head(10)

,customer_id,address,address_flag
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [2]:
df_sales_amount = df_receipt[['customer_id', 'amount']]. \
                        groupby('customer_id').sum().reset_index()
pct25 = np.quantile(df_sales_amount['amount'], 0.25)
pct50 = np.quantile(df_sales_amount['amount'], 0.5)
pct75 = np.quantile(df_sales_amount['amount'], 0.75)

def pct_group(x):
    if x < pct25:
        return 1
    elif pct25 <= x < pct50:
        return 2
    elif pct50 <= x < pct75:
        return 3
    elif pct75 <= x:
        return 4

df_sales_amount['pct_group'] = df_sales_amount['amount'].apply(lambda x: pct_group(x))
df_sales_amount.head(10)

,customer_id,amount,pct_group
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [15]:
df_age = pd.concat([df_customer[['customer_id','birth_day']],
                    df_customer['age'].apply(lambda x: min(math.floor(x / 10) * 10,60))
                   ],axis=1)
df_age.head(10)

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [31]:
df_gender_age = pd.concat([df_age,df_customer[['gender','gender_cd']]],axis=1)
df_gender_age['gender_age'] = df_gender_age['age'] + df_gender_age['gender_cd'].astype('int')
df_gender_age.head(10)

,customer_id,birth_day,age,gender,gender_cd,gender_age
0,CS021313000114,1981-04-29,30,女性,1,31
1,CS037613000071,1952-04-01,60,不明,9,69
2,CS031415000172,1976-10-04,40,女性,1,41
3,CS028811000001,1933-03-27,60,女性,1,61
4,CS001215000145,1995-03-29,20,女性,1,21
5,CS020401000016,1974-09-15,40,男性,0,40
6,CS015414000103,1977-08-09,40,女性,1,41
7,CS029403000008,1973-08-17,40,男性,0,40
8,CS015804000004,1931-05-02,60,男性,0,60
9,CS033513000180,1962-07-11,50,女性,1,51


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [32]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], 
               columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [8]:
df_sales_sum = df_receipt.query("not customer_id.str.startswith('Z')",engine='python').groupby('customer_id').amount.sum().reset_index()
df_sales_sum['amount_ss'] = preprocessing.scale(df_sales_sum['amount'])
df_sales_sum.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [10]:
df_sales_amount = df_receipt.query("not customer_id.str.startswith('Z') ",engine='python').groupby('customer_id').amount.sum().reset_index()
df_sales_amount['amount_ss'] = preprocessing.minmax_scale(df_sales_amount['amount'])
df_sales_amount.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [11]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")', 
                                   engine='python'). \
                                   groupby('customer_id'). \
                                   agg({'amount':'sum'}).reset_index()
df_sales_amount['amount_log10'] = np.log10(df_sales_amount['amount'] + 1)
df_sales_amount.head(10)

,customer_id,amount,amount_log10
0,CS001113000004,1298,3.113609
1,CS001114000005,626,2.797268
2,CS001115000010,3044,3.483587
3,CS001205000004,1988,3.298635
4,CS001205000006,3337,3.523486
5,CS001211000025,456,2.659916
6,CS001212000027,448,2.652246
7,CS001212000031,296,2.472756
8,CS001212000046,228,2.359835
9,CS001212000070,456,2.659916


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [12]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")', 
                                   engine='python'). \
                                   groupby('customer_id'). \
                                   agg({'amount':'sum'}).reset_index()
df_sales_amount['amount_log'] = np.log(df_sales_amount['amount'] + 1)
df_sales_amount.head(10)

,customer_id,amount,amount_log
0,CS001113000004,1298,7.169350
1,CS001114000005,626,6.440947
2,CS001115000010,3044,8.021256
3,CS001205000004,1988,7.595387
4,CS001205000006,3337,8.113127
5,CS001211000025,456,6.124683
6,CS001212000027,448,6.107023
7,CS001212000031,296,5.693732
8,CS001212000046,228,5.433722
9,CS001212000070,456,6.124683


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [13]:
df_profit = df_product[['unit_price','unit_cost']].copy()
df_profit['profit'] = df_profit['unit_price'] - df_profit['unit_cost']
df_profit.head(10)

,unit_price,unit_cost,profit
0,198.0,149.0,49.0
1,218.0,164.0,54.0
2,230.0,173.0,57.0
3,248.0,186.0,62.0
4,268.0,201.0,67.0
5,298.0,224.0,74.0
6,338.0,254.0,84.0
7,420.0,315.0,105.0
8,498.0,374.0,124.0
9,580.0,435.0,145.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [17]:
df_profit = df_product[['unit_price','unit_cost']].copy()
df_profit['profit_rate'] = (df_product['unit_price'] - df_product['unit_cost']) / df_product['unit_price']
df_profit['profit_rate'].mean(skipna=True)

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [4]:
df_sales_amount = df_product[['unit_price','unit_cost']].copy()
df_sales_amount['new_price']  = df_sales_amount['unit_cost'].apply(lambda x: np.floor(x / 0.7))
df_sales_amount['profit_rate'] = (df_sales_amount['new_price'] - df_sales_amount['unit_cost']) \
                                    / df_sales_amount['new_price']
df_sales_amount.head(10)

,unit_price,unit_cost,new_price,profit_rate
0,198.0,149.0,212.0,0.297170
1,218.0,164.0,234.0,0.299145
2,230.0,173.0,247.0,0.299595
3,248.0,186.0,265.0,0.298113
4,268.0,201.0,287.0,0.299652
5,298.0,224.0,320.0,0.300000
6,338.0,254.0,362.0,0.298343
7,420.0,315.0,450.0,0.300000
8,498.0,374.0,534.0,0.299625
9,580.0,435.0,621.0,0.299517


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [7]:
df_sales_amount = df_product.copy()
df_sales_amount['new_price'] = df_sales_amount['unit_cost'].apply(lambda x: np.round(x/0.7))
df_sales_amount['profit_rate'] = (df_sales_amount['new_price'] - df_sales_amount['unit_cost']) \
                                    / df_sales_amount['new_price']
df_sales_amount.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,profit_rate
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [8]:
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.ceil(x/0.7))
df_tmp['profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,profit_rate
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,235.0,0.302128
2,P040101003,04,0401,040101,230.0,173.0,248.0,0.302419
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,288.0,0.302083
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,451.0,0.301552
8,P040101009,04,0401,040101,498.0,374.0,535.0,0.300935
9,P040101010,04,0401,040101,580.0,435.0,622.0,0.300643


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [9]:
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_price'].apply(lambda x: np.floor(x * 1.1))
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price
0,P040101001,04,0401,040101,198.0,149.0,217.0
1,P040101002,04,0401,040101,218.0,164.0,239.0
2,P040101003,04,0401,040101,230.0,173.0,253.0
3,P040101004,04,0401,040101,248.0,186.0,272.0
4,P040101005,04,0401,040101,268.0,201.0,294.0
5,P040101006,04,0401,040101,298.0,224.0,327.0
6,P040101007,04,0401,040101,338.0,254.0,371.0
7,P040101008,04,0401,040101,420.0,315.0,462.0
8,P040101009,04,0401,040101,498.0,374.0,547.0
9,P040101010,04,0401,040101,580.0,435.0,638.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [24]:
df_tmp = pd.merge(df_receipt,df_product,how='inner',on='product_cd')

df_sum = df_tmp.groupby('customer_id').amount.sum().reset_index()
df_sum2 = df_tmp.query('category_major_cd == "07"').groupby('customer_id').amount.sum().reset_index()
df_rate = pd.merge(df_sum,df_sum2,how='inner',on='customer_id')
df_rate['rate_code_7'] = df_rate['amount_y'] / df_rate['amount_x']
df_rate.head(10)

,customer_id,amount_x,amount_y,rate_code_7
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [13]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp['process_date'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str')) - pd.to_datetime(df_tmp['application_date'])
df_tmp.head(10)

,customer_id,sales_ymd,application_date,process_date
0,CS006214000001,20181103,20150201,1371 days
1,CS006214000001,20170509,20150201,828 days
2,CS006214000001,20170608,20150201,858 days
3,CS006214000001,20170608,20150201,858 days
4,CS006214000001,20181028,20150201,1365 days
5,CS006214000001,20181028,20150201,1365 days
6,CS006214000001,20170509,20150201,828 days
7,CS006214000001,20190908,20150201,1680 days
8,CS006214000001,20180131,20150201,1095 days
9,CS006214000001,20170705,20150201,885 days


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [20]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapse_date'] = df_tmp[['sales_ymd','application_date']].apply(lambda x: relativedelta(x[0],x[1]).years * 12 + relativedelta(x[0],x[1]).months,axis=1)
df_tmp.sort_values('customer_id').head(10)

,customer_id,sales_ymd,application_date,elapse_date
60376,CS001113000004,2019-03-08,2015-11-05,40
60377,CS001113000004,2019-03-08,2015-11-05,40
20159,CS001114000005,2019-07-31,2016-04-12,39
20157,CS001114000005,2018-05-03,2016-04-12,24
20156,CS001114000005,2018-05-03,2016-04-12,24
20158,CS001114000005,2019-07-31,2016-04-12,39
29145,CS001115000010,2017-12-28,2015-04-17,32
29144,CS001115000010,2018-07-01,2015-04-17,38
29143,CS001115000010,2019-04-05,2015-04-17,47
29142,CS001115000010,2017-12-28,2015-04-17,32


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [14]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_year'] = df_tmp[['sales_ymd','application_date']].apply(lambda x: relativedelta(x[0],x[1]).years,axis=1)
df_tmp.query('elapsed_year > 0').head(10)

,customer_id,sales_ymd,application_date,elapsed_year
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
3,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
5,CS006214000001,2018-10-28,2015-02-01,3
6,CS006214000001,2017-05-09,2015-02-01,2
7,CS006214000001,2019-09-08,2015-02-01,4
8,CS006214000001,2018-01-31,2015-02-01,2
9,CS006214000001,2017-07-05,2015-02-01,2


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [33]:
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],how='inner',on='customer_id')
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] = \
    (df_tmp['sales_ymd'].view(np.int64) / 10**9) - (df_tmp['application_date'].\
                                                    view(np.int64) / 10**9)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
3,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
5,CS006214000001,2018-10-28,2015-02-01,117936000.0
6,CS006214000001,2017-05-09,2015-02-01,71539200.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [30]:
from datetime import timedelta
df_tmp = df_receipt[['customer_id','sales_ymd']].copy()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['elapsed_day'] = df_tmp['sales_ymd'].apply(lambda x: x.weekday())
df_tmp['week_start_date'] = df_tmp[['sales_ymd','elapsed_day']]\
                                .apply(lambda x: x[0] - timedelta(days=x[1]),axis=1)
df_tmp[['customer_id','sales_ymd','week_start_date','elapsed_day']].head(10)

,customer_id,sales_ymd,week_start_date,elapsed_day
0,CS006214000001,2018-11-03,2018-10-29,5
1,CS008415000097,2018-11-18,2018-11-12,6
2,CS028414000014,2017-07-12,2017-07-10,2
3,ZZ000000000000,2019-02-05,2019-02-04,1
4,CS025415000050,2018-08-21,2018-08-20,1
5,CS003515000195,2019-06-05,2019-06-03,2
6,CS024514000042,2018-12-05,2018-12-03,2
7,CS040415000178,2019-09-22,2019-09-16,6
8,ZZ000000000000,2017-05-04,2017-05-01,3
9,CS027514000015,2019-10-10,2019-10-07,3


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [34]:
df_customer.sample(int(len(df_customer) * 0.01)).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
19340,CS003315000021,大塚 まひる,1,女性,1983-08-27,35,201-0002,東京都狛江市東野川**********,S13003,20150312,0-00000000-0
20627,CS016613000087,田辺 綾女,1,女性,1951-05-21,67,184-0004,東京都小金井市本町**********,S13016,20141208,0-00000000-0
3804,CS013513000158,宮地 奈月,1,女性,1959-09-27,59,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150425,0-00000000-0
1293,CS005403000059,杉田 宏行,0,男性,1972-12-19,46,167-0032,東京都杉並区天沼**********,S13005,20170703,0-00000000-0
4347,CS005412000214,上原 あい,1,女性,1969-07-04,49,176-0002,東京都練馬区桜台**********,S13005,20160826,0-00000000-0
2937,CS009214000004,村山 勇,1,女性,1997-05-04,21,158-0081,東京都世田谷区深沢**********,S13009,20150312,F-20100902-E
5056,CS023415000030,石倉 路子,1,女性,1976-03-17,43,212-0016,神奈川県川崎市幸区南幸町**********,S14023,20141214,B-20100122-C
20652,CS019415000302,宮崎 華子,1,女性,1972-09-13,46,176-0006,東京都練馬区栄町**********,S13019,20170518,0-00000000-0
5372,CS029411000045,貫地谷 ひかり,1,女性,1971-09-09,47,133-0065,東京都江戸川区南篠崎町**********,S12029,20150513,B-20100516-A
1713,CS026515000092,中田 涼,1,女性,1960-09-29,58,251-0047,神奈川県藤沢市辻堂**********,S14026,20150605,E-20100717-F


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別ごとに件数を集計せよ。

In [56]:
_, df_tmp = train_test_split(df_customer, test_size=0.1, 
                             stratify=df_customer['gender_cd'])
df_tmp.groupby('gender_cd').agg({'customer_id' : 'count'})

,customer_id
gender_cd,
0,298
1,1793
9,107


---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [4]:
df_tmp = df_receipt.query("not customer_id.str.startswith('Z')",engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_ss'] = preprocessing.scale(df_tmp['amount'])
df_tmp.query('abs(amount_ss) >= 3',engine='python').head(10)

,customer_id,amount,amount_ss
332,CS001605000009,18925,6.019921
1755,CS006415000147,12723,3.740202
1817,CS006515000023,18372,5.816651
1833,CS006515000125,12575,3.685800
1841,CS006515000209,11373,3.243972
1870,CS007115000006,11528,3.300946
1941,CS007514000056,13293,3.949721
1943,CS007514000094,15735,4.847347
1951,CS007515000107,11188,3.175970
1997,CS007615000026,11959,3.459372


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [30]:
df_tmp = df_receipt.query("not customer_id.str.startswith('Z')",engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp2 = df_tmp['amount'].quantile(q=[0.25,0.75])
iqr = df_tmp2[0.75] - df_tmp2[0.25]
df_tmp.query('amount < @df_tmp2[0.25] - 1.5 * @iqr or amount > @df_tmp2[0.75] + 1.5 * @iqr',engine='python').head(10)

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [32]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [34]:
df_product1 = df_product.dropna(how='any')
df_product1.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [21]:
df_unit_price_mean = int(df_product['unit_price'].mean())
df_unit_cost_mean = int(df_product['unit_cost'].mean())
df_product_2 = df_product.fillna({'unit_price':df_unit_price_mean,'unit_cost':df_unit_cost_mean})
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [24]:
df_product3 = df_product.fillna({'unit_price': np.round(df_product['unit_price'].median()),'unit_cost': np.round(df_product['unit_cost'].median())})
df_product3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [21]:
df_tmp = df_product.groupby('category_small_cd').agg({'unit_price': 'median','unit_cost': 'median'}).reset_index()
df_tmp.columns = ['category_small_cd','median_price','median_cost']
df_product_4 = pd.merge(df_tmp,df_product,how='inner',on = 'category_small_cd')

df_product_4 = df_product_4.fillna({'unit_price': df_product_4['median_price'], 'unit_cost': df_product_4['median_cost']})
df_product_4.isnull().sum()

category_small_cd     0
median_price          0
median_cost           0
product_cd            0
category_major_cd     0
category_medium_cd    0
unit_price            0
unit_cost             0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [60]:
df_tmp = pd.merge(df_customer,df_receipt,how='inner',on='customer_id').fillna({'amount':0})

df_tmp_1= df_tmp.groupby('customer_id').amount.sum().reset_index()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp_2 = df_tmp.query('sales_ymd.dt.year == 2019',engine='python').groupby('customer_id').amount.sum().reset_index().rename(columns = {'amount':'2019_amount'})

df_tmp = pd.merge(df_tmp_1,df_tmp_2,how='inner',on='customer_id')
df_tmp['amount_rate'] = df_tmp['2019_amount'] / df_tmp['amount']
df_tmp.query('amount_rate > 0').head(10)

,customer_id,amount,2019_amount,amount_rate
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,188,0.300319
2,CS001115000010,3044,578,0.189882
3,CS001205000004,1988,702,0.353119
4,CS001205000006,3337,486,0.145640
5,CS001211000025,456,456,1.000000
6,CS001212000070,456,456,1.000000
7,CS001214000009,4685,664,0.141729
8,CS001214000017,4132,2962,0.716844
9,CS001214000048,2374,1889,0.795703


---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [81]:
df_customer_1 = pd.merge(df_customer[['postal_cd','customer_id']],df_geocode[['postal_cd','longitude','latitude']],how='inner',on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id').agg({'longitude':'mean','latitude':'mean'}).reset_index().rename(columns={'longitude':'m_longitude','latitude':'m_latitude'})
df_customer_1 = pd.merge(df_customer,df_customer_1,how='inner',on='customer_id')
df_customer_1.head(3)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,m_longitude,m_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.68965,35.67374


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [91]:
def calc_distance(long1,lat1,long2,lat2):
    return 6371 * math.acos(
                    np.sin(np.radians(long1)) * np.sin(np.radians(long2)) + np.cos(np.radians(long1)) * np.cos(np.radians(long2)) * np.cos(np.radians(lat1 - lat2))
    )
df_tmp = pd.merge(df_customer_1,df_store,how='inner',left_on='application_store_cd',right_on='store_cd')
df_tmp['distance'] = df_tmp[['m_longitude', 'm_latitude','longitude', 'latitude']].apply(lambda x: calc_distance(x[0],x[1],x[2],x[3]),axis=1)
df_tmp.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address_x,application_store_cd,application_date,...,store_name,prefecture_cd,prefecture,address_y,address_kana,tel_no,longitude,latitude,floor_area,distance
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.140524
1,CS021313000025,砂川 あさみ,1,女性,1981-10-05,37,259-1131,神奈川県伊勢原市伊勢原**********,S14021,20150326,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,0.367060
2,CS021411000096,布施 杏,1,女性,1971-10-30,47,259-1114,神奈川県伊勢原市高森**********,S14021,20160621,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,2.153135
3,CS021415000150,早美 紗季,1,女性,1969-11-26,49,259-1141,神奈川県伊勢原市上粕屋**********,S14021,20150601,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,2.956009
4,CS021313000046,会田 薫,9,不明,1983-01-19,36,259-1144,神奈川県伊勢原市池端**********,S14021,20150722,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.341027
5,CS021103000002,庄司 染五郎,0,男性,2006-11-24,12,259-1111,神奈川県伊勢原市西富岡**********,S14021,20150317,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,2.099841
6,CS021214000028,吉井 菜摘,1,女性,1991-07-12,27,259-1132,神奈川県伊勢原市桜台**********,S14021,20151007,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.115342
7,CS021512000095,佐々木 陽子,1,女性,1963-01-22,56,259-1126,神奈川県伊勢原市沼目**********,S14021,20150820,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,2.318651
8,CS021613000002,内村 貴美子,1,女性,1950-02-18,69,259-1132,神奈川県伊勢原市桜台**********,S14021,20151125,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.115342
9,CS021412000147,三輪 さやか,1,女性,1971-08-23,47,259-1103,神奈川県伊勢原市三ノ宮**********,S14021,20150930,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,4.295739


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！